# CoreLogic

## 1. Explore and extract the data

This notebook is for UM associated studnets, faculty and researchers who have been given access to CoreLogic real estate data. The files are located here (for Mac) smb://ulib-licensed-content.m.storage.umich.edu/ulib-licensed-content/corelogic and here (for windows) \\ulib-licensed-content.m.storage.umich.edu\ulib-licensed-content\corelogic. Mount the drive. 

The associated excel files give you inforamtion about the format of data. The data is at the level of a parcel, so you can aggregate to any spatial unit bigger than that, such as a census block or tract. There are three different data: tax, deed sales, and foreclosure. Read the metadata excel files associated with each type of data to know what information is included. Spatial location is identified with a point, which is likely the centroid of a given parcel.  

Each file is line-based. In Python, if a file in line-based, the file object is a lazy generator of lines so you can read the file line-by-line. 

Let us first look at the structure of the data. You do not need to unzip the data, most modern languages can read the data from a zipped file on the fly. If you do not have the required modules, install them using conda or pip. I have gotten surprising results in trying to install from within Jupyter, but if you want to try read the following https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

In [10]:
import os
import zipfile
import pandas as pd
import numpy as np
import geopandas as gp
import shapely as sh
import fiona
os.chdir('/Volumes/corelogic')
os.listdir()

['Michigan_Uni_Tax_AKZA_85HRQ5_Counts.xlsx',
 'Tax_Layout_w_Property_Level_lat_long_w_code_01262017.xlsx',
 'Deed_Layout_PropertyLevel_Lat_Long_11172016.xlsx',
 'Michigan_Uni_FCL_AKZA_85HRYY_Data.zip',
 'FCL_Layout_Bulk_w_Code_Tables_04072017.xlsx',
 'Michigan_Uni_Deed_KZA_85HRZB_Count.xlsx',
 'Michigan_Uni_FCL_AKZA_85HRYY_Count.xlsx',
 'Michigan_Uni_Tax_AKZA_85HRQ5.zip',
 'Michigan_Uni_Deed_KZA_85HRZB.zip']

In [15]:
# Let us see the header for a file and a couple of sample data row
dataFile = ['Michigan_Uni_Tax_AKZA_85HRQ5.zip', 'Michigan_Uni_FCL_AKZA_85HRYY_Data.zip', 'Michigan_Uni_Deed_KZA_85HRZB.zip']   

dataFileIndex = 0   # File index goes here, choose 0 for tax, 1 for foreclosure, and 2 for deed. All the subsequent will be based on this step. 
count = 1
sampleList = []
with zipfile.ZipFile(dataFile[dataFileIndex], 'r') as z:            
    fileList = z.infolist()
    with z.open(fileList[0], 'r') as f:
        for line in f:
            line = line.decode(encoding='utf-8', errors='strict')
            intermediary_object=line.strip().split("|")
            sampleList.append(intermediary_object)
            count=count+1
            if count==10:
               break

# Let us display column headings and the data, 
print(sampleList[0])
print('THERE ARE %d FIELDS' % len(sampleList[0]))

['FIPS CODE', 'UNFORMATTED APN', 'APN SEQUENCE NBR', 'FORMATTED APN', 'ORIGINAL APN', 'PREVIOUS PARCEL NUMBER', 'P-ID-IRIS-FRMTD', 'ACCOUNT NUMBER', 'MAP REFERENCE1', 'MAP REFERENCE2', 'CENSUS TRACT', 'BLOCK NUMBER', 'LOT NUMBER', 'RANGE', 'TOWNSHIP', 'SECTION', 'QUARTER SECTION', 'FLOOD ZONE COMMUNITY PANEL ID', 'LAND USE', 'COUNTY USE1', 'COUNTY USE2', 'MOBILE HOME IND', 'ZONING', 'PROPERTY INDICATOR', 'MUNICIPALITY NAME', 'MUNICIPALITY CODE', 'SUBDIVISION TRACT NUMBER', 'SUBDIVISION PLAT BOOK', 'SUBDIVISION PLAT PAGE', 'SUBDIVISION NAME', 'PROPERTY LEVEL LATITUDE', 'PROPERTY LEVEL LONGITUDE', 'SITUS HOUSE NUMBER PREFIX', 'SITUS HOUSE NUMBER', 'SITUS HOUSE NUMBER #2', 'SITUS HOUSE NUMBER SUFFIX', 'SITUS DIRECTION', 'SITUS STREET NAME', 'SITUS MODE', 'SITUS QUADRANT', 'SITUS UNIT NUMBER', 'SITUS CITY', 'SITUS STATE', 'SITUS ZIP CODE', 'SITUS CARRIER CODE', 'OWNER CORPORATE INDICATOR', 'OWNER FULL NAME', 'OWNER1 LAST NAME', 'OWNER1 FIRST NAME & MI', 'OWNER2 LAST NAME', 'OWNER2 FIRST NA

In [12]:
#Display sample records 
df1 = pd.DataFrame(sampleList[1:9], columns = sampleList[0]) 
pd.set_option('display.max_columns', None)
df1
# Or print it to see 
#print(df1)

,FIPS,APN (Parcel Number) (unformatted),PCL ID IRIS FORMATTED,APN SEQUENCE NUMBER,PENDING RECORD INDICATOR,CORPORATE INDICATOR,OWNER FULL NAME,OWNER 1 LAST NAME,OWNER 1 FIRST NAME & M I,OWNER 2 LAST NAME,OWNER 2 FIRST NAME & MI,OWNER ETAL INDICATOR,C/O NAME,OWNER RELATIONSHIP RIGHTS CODE,OWNER RELATIONSHIP TYPE,PARTIAL INTEREST INDICATOR,ABSENTEE OWNER STATUS,PROPERTY LEVEL LATITUDE,PROPERTY LEVEL LONGITUDE,SITUS HOUSE NUMBER PREFIX,SITUS HOUSE NUMBER,SITUS HOUSE NUMBER SUFFIX,SITUS DIRECTION,SITUS STREET NAME,SITUS MODE,SITUS QUADRANT,SITUS APARTMENT UNIT,SITUS CITY,SITUS STATE,SITUS ZIP CODE,SITUS CARRIER CODE,MAILING HOUSE NUMBER PREFIX,MAILING HOUSE NUMBER,MAILING HOUSE NUMBER SUFFIX,MAILING DIRECTION,MAILING STREET NAME,MAILING MODE,MAILING QUADRANT,MAILING APARTMENT UNIT,MAILING PROPERTY CITY,MAILING PROPERTY STATE,MAILING PROPERTY ADDRESS ZIP CODE,MAILING CARRIER CODE,BATCH-ID,BATCH-SEQ,MULTI APN,SELLER LAST NAME,SELLER FIRST NAME,SELLER NAME 1,SELLER NAME 2,SALE CODE,SALE AMOUNT,SALE DATE (YYYYMMDD),RECORDING DATE (YYYYMMDD),DOCUMENT TYPE,TRANSACTION TYPE,DOCUMENT NUMBER,BOOK/PAGE (6x6),LENDER FULL NAME,LENDER LAST NAME,LENDER FIRST NAME,LENDER ADDRESS,LENDER CITY,LENDER ST,LENDER ZIP,LENDER COMPANY CODE,TITLE COMPANY NAME,TITLE COMPANY CODE,MORTGAGE AMOUNT,MORTGAGE DATE,MORTGAGE INTEREST RATE,MORTGAGE LOAN TYPE CODE,MORTGAGE DEED TYPE,MORTGAGE TERM CODE,MORTGAGE TERM,MORTGAGE DUE DATE,MORTGAGE ASSUMPTION AMOUNT,MTG SEQ NUMBER,PRI-CAT-CODE,MTG SEC CAT CODES 1X10,DEED SEC CAT CODES 2X10,OWNERSHIP TRANSFER PERCENTAGE,LAND USE,PROPERTY INDICATOR,SELLER CARRY BACK,INTER FAMILY,PRIVATE PARTY LENDER,MORTGAGE INTEREST RATE TYPE,CONSTRUCTION LOAN,RESALE/NEW CONSTRUCTION,FORECLOSURE,CASH/MORTGAGE PURCHASE,EQUITY FLAG,REFI FLAG,RESIDENTIAL MODEL INDICATOR,ADD/CHANGE FIELD,FILLER
0,01001,06010100010050000,06-01-01-0-001-005-000-0,1,,Y,CARTER,,,,,0,MONA C MEREDITH,LP,,,,32.660776,-86.717423,,,,,,,,,BILLINGSLEY,AL,36006,,,3914,,,MERRIFIELD,DR,,,VALLEY GRANDE,AL,367032729,C020,19300101,8,,OWNER,RECORD,OWNER RECORD,,,,19500424,,DD,1,,,,,,,,,,,,,,,,,,,,,,,A,,MV,,163,10,,False,,,,M,,,,,Y,,
1,01001,06010100110010000,06-01-01-0-011-001-000-0,1,,Y,CARTER,,,,,0,MONA C MEREDITH,LP,,,,32.659266,-86.718646,,,,,,,,,BILLINGSLEY,AL,36006,,,3914,,,MERRIFIELD,DR,,,VALLEY GRANDE,AL,367032729,C020,19300101,9,,OWNER,RECORD,OWNER RECORD,,,,19500530,,DD,1,,,,,,,,,,,,,,,,,,,,,,,A,,M,,465,80,,False,,,,M,,,,,,,
2,01001,14092900000010000,14-09-29-0-000-001-000-0,1,,,BRUCE SARA,BRUCE,SARA,,,0,,,,,,32.515287,-86.888660,,,,,OFF HWY 14,,,,PRATTVILLE,AL,36067,,,4702,,,PINERIDGE,CIR,SE,,DECATUR,AL,356035117,R007,19300101,10,,OWNER,RECORD,OWNER RECORD,,,,19701221,,DD,1,,,,,,,,,,,,,,,,,,,,,,,A,,M,,326,50,,False,,,,M,,,,,,,
3,01001,06010100180020000,06-01-01-0-018-002-000-0,1,,,CARTER RUTH,CARTER,RUTH,,,0,LEE NUGENT,,,,,32.655807,-86.719932,,,,,COUNTY ROAD 37,,,,BILLINGSLEY,AL,360063525,R501,,29,,,TREASURE ISLE,,,,,,70461,,19300101,11,,OWNER,RECORD,OWNER RECORD,,,,19500424,,DD,1,,,,,,,,,,,,,,,,,,,,,,,A,,MV,,100,10,,False,,,,M,,,,,Y,,
4,01001,19061410010010000,19-06-14-1-001-001-000-0,1,,,THAMES-ARONOV PVILLE E,THAMES-ARONOV,PVILLE E,,,0,,,,,,32.458475,-86.420017,,,,,,,,,,AL,,,,,,,PO BOX 235000,,,,MONTGOMERY,AL,361235000,B099,19300101,12,,OWNER,RECORD,OWNER RECORD,,,,19780313,,DD,1,,,,,,,,,,,,,,,,,,,,,,,A,,M,,,00,,False,,,,M,,,,,,,
5,01001,01093200000020000,01-09-32-0-000-002-000-0,1,,,BROONER CLIFFORD E & JEAN S,BROONER,CLIFFORD E,BROONER,JEAN S,0,,,,,,32.676209,-86.488168,,,,,US HIGHWAY 31,,N,,MARBURY,AL,360513003,R801,,2856,,N,HWY 31,,,,,,36051,,19300101,13,,OWNER,RECORD,OWNER RECORD,,,,19621108,,DD,1,,,,,,,,,,,,,,,,,,,,,,,A,,M,,400,80,,False,,,,M,,,,,,,
6,01001,11030600000390000,11-03-06-0-000-039-000-0,1,,,BROWN CHARLIE,BROWN,CHARLIE,,,0,MARTHA BROWN,,,,,32.569739,-86.595967,,,,,COUNTY ROAD 21,,N,,PRATTVILLE,AL,36067,,,970,,,COUNTY ROAD 40,,W,,PRATTVILLE,AL,360677431,R003,19300101,14,,OWNER,RECORD,OWNER RECORD,,,,19160721,,DD,1,,,,,,,,,,,,,,,,,,,,,,,A,,M,,500,7

Most fields in the data are empty. Also see that sale date is available but sale amount is missing. Before planning any anlysis, please confirm that you have sufficient data. In most cases you do not want all the fields. Let us extract some fields.

In [14]:
# Create a function to search field strings 
def searchField(fieldString):
    strIndex = [i for i in sampleList[0] if fieldString in i.lower()] 
    indexLat = [sampleList[0].index(i) for i in strIndex]
    print((strIndex,indexLat))

# For tax data: Find location index of latitude, longitude, zip code, census tract, total value etc. from the Tax file 
if dataFileIndex == 0:
    fieldNameStr = ['apn', 'fips', 'latitude', 'longitude', 'zip code', 'census tract', 'total value', 'number of units', 'square feet']
elif dataFileIndex == 2:
# For deed data: Find location index of latitude, longitude, zip code, sale date, and sale amount etc.  
    fieldNameStr = ['apn', 'fips','census tract', 'latitude', 'longitude', 'sale date', 'sale amount', 'property indicator']
else:
    print('Only 0 or 2 are valid no. You can add 1, to get foreclosure, to the loop if you want')
indexList = [searchField(i) for i in fieldNameStr]

(['UNFORMATTED APN', 'APN SEQUENCE NBR', 'FORMATTED APN', 'ORIGINAL APN', 'MULTI APN FLAG'], [1, 2, 3, 4, 91])
(['FIPS CODE'], [0])
(['PROPERTY LEVEL LATITUDE'], [30])
(['PROPERTY LEVEL LONGITUDE'], [31])
(['SITUS ZIP CODE', 'MAIL ZIP CODE'], [43, 67])
(['CENSUS TRACT'], [10])
(['TOTAL VALUE CALCULATED', 'TOTAL VALUE CALCULATED IND', 'ASSD TOTAL VALUE', 'MKT TOTAL VALUE', 'APPR TOTAL VALUE'], [70, 73, 76, 79, 82])
(['NUMBER OF UNITS'], [169])
(['UNIVERSAL BUILDING SQUARE FEET', 'BUILDING SQUARE FEET IND', 'BUILDING SQUARE FEET', 'LIVING SQUARE FEET', 'GROUND FLOOR SQUARE FEET', 'GROSS SQUARE FEET', 'ADJUSTED GROSS SQUARE FEET', 'BASEMENT SQUARE FEET', 'GARAGE/PARKING SQUARE FEET'], [121, 122, 123, 124, 125, 126, 127, 128, 129])


It seems to me that APN is the common field across the three different data, so if you want to link subsets from the three data you will have to use APN to join the data, but APN may be unique only within a county. Only two of the three files have latitude and longitude. 

Now let us extract some data. Note that for the 'tax data' the first column in the data is the FIPS code of the county. This  is a five digit code with the first two uniquely identifying the state and the next three the county in the state. 

Let us extract a part of the data for the District of Columbia (fips code 11). This is a small state and we will be able to work with it in the course of this workshop.


In [5]:
#Let us open a file for writing and create a header row

with open ("output.csv",'w') as resultFile:
    if dataFileIndex == 0:
        resultFile.write('fips code, apn, census tract, latitude, longitude, zip code, market value, square feet, no of units')
    elif dataFileIndex == 2:
        resultFile.write('fips code, apn, latitude, longitude, sale amount, sale date, property indicator')
    else:
        print ('Wrong file index')
    resultFile.write('\n')

# Now append the data 
count = 1
with open("output.csv",'a') as resultFile:
    with zipfile.ZipFile(dataFile[dataFileIndex], 'r') as z:
        fileList = z.infolist()
        with z.open(fileList[0], 'r') as f:
                for _ in range(1):
                    next(f)
                for line in f:
                    line = line.decode(encoding='utf-8', errors='strict')
                    line = line.strip().replace('|', ',')                
                    line=line.split(",")
                    line = ['NA' if x == '' else x for x in line]   # You could use replace in the string, but just for fun 
                    if line[0][:2] == '11': #'26161':
                        if dataFileIndex == 0:
                            relevantData = [line[0]] + [line[10]] + line[30:32] + [line[43]] + [line[79]] + [line[121]] + [line[169]]
                        elif dataFileIndex == 2:
                            relevantData = line[0:2] + line[17:19] + line[51:53] + [line[83]]
                        else:
                            print ('Error')
                        relevantData = ','.join(relevantData)
                        resultFile.write(relevantData)
                        resultFile.write('\n')
                        count=count+1
                    if count==45000:
                        break

## 2. Spatialize the data
Now let us create a spatial data from the text file we saved. Geopandas is built on top of pandas dataframe and has additional structure to accomodate geometric and geographic details. 

We want to make sure that we have lat long data to fix the location

In [6]:
df = pd.read_csv("output.csv", skipinitialspace=True)  #Press shift-tab to see the arguments, sometimes leading white space can create problems
print(df.head())

# Are there NaN in latitude and longitude
index = df.latitude.isnull()
print(df[index])

# Remove NaNs
index = df['latitude'].notnull() 
df = df[index]

   fips code           apn   latitude  longitude sale amount  sale date  \
0      11001  0013    2005  38.905551 -77.053868   139500.00        NaN   
1      11001  0013    2006  38.905551 -77.053868    92500.00        NaN   
2      11001  0013    2009  38.905551 -77.053868   155000.00        NaN   
3      11001  0013    2011  38.905551 -77.053868   142500.00        NaN   
4      11001  0013    2013  38.905551 -77.053868   178000.00        NaN   

   property indicator  
0                11.0  
1                11.0  
2                11.0  
3                11.0  
4                11.0  
       fips code  apn  latitude  longitude sale amount  sale date  \
253        11001  NaN       NaN        NaN    80500.00        NaN   
316        11001  NaN       NaN        NaN   200000.00        NaN   
499        11001  NaN       NaN        NaN   300100.00        NaN   
537        11001  NaN       NaN        NaN   120000.00        NaN   
660        11001  NaN       NaN        NaN    44800.00      

In [7]:
df.index[2:4]

Int64Index([2, 3], dtype='int64')

In [8]:
# Check the type of latitude and longitude
print(type(df['latitude'].iloc[0]))           # It coule be a string, so make sure we convert it to numeric

geomCol = [sh.geometry.Point (x, y) for x, y in zip(pd.to_numeric(df['longitude']), pd.to_numeric(df['latitude']))]

<class 'numpy.float64'>


In [9]:
# Now we are good to go
geomCol = [sh.geometry.Point (x, y) for x, y in zip(pd.to_numeric(df['longitude']), pd.to_numeric(df['latitude']))]
if dataFileIndex == 0:
    geoDf = gp.GeoDataFrame(df[['fips code', 'census tract', 'zip code', 'market value', 'square feet', 'no of units']], geometry = geomCol)
elif dataFileIndex == 2:
    geoDf = gp.GeoDataFrame(df[['fips code', 'apn', 'sale amount', 'sale date', 'property indicator']], geometry = geomCol)
geoDf.crs = {'init' :'epsg:4326'}

# Write the data to shapefile or geojson, open it is QGIS and examine
geoDf.head()
geoDf.to_file('dc_prop_sale.shp')